# Advent of code - Day3
https://adventofcode.com/2023/day/3

In [10]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', True)

import re

# Part 1

In [102]:
def read_matrix_from_file(file_path):
    with open(file_path, 'r') as file:
        input_text = file.read()
        lines = input_text.strip().split('\n')
        matrix = [list(line) for line in lines]
        numpy_matrix = np.array(matrix)
        return numpy_matrix

engine_schematic = read_matrix_from_file("day3_input.txt")
special_characters = set("!@#$%^&*()_+{}[]:;<>,?/\\|=~-")

def check_symbols(x, y, engine_schematic):
    x_max = engine_schematic.shape[0]
    y_max = engine_schematic.shape[1]

    x_upper = x+2 if x+2 <= x_max else x+1
    y_upper = y+2 if y+2 <= y_max else y+1
    x_lower = x-1 if x >= 1 else x
    y_lower = y-1 if y >= 1 else y

    neighbors = set(engine_schematic[x_lower:x_upper, y_lower:y_upper].flatten())
    # print(engine_schematic[x_lower:x_upper, y_lower:y_upper])
    if len(neighbors&special_characters)>0:
        # print(neighbors&special_characters)
        return True
    else:
        return False
    
def check_full_number_symbol(current_digit, x, y, engine_schematic):
    if len(current_digit) == 1:
        return check_symbols(x, y, engine_schematic)
    elif len(current_digit) == 2:
        return check_symbols(x, y, engine_schematic) or  check_symbols(x, y-1, engine_schematic)
    elif len(current_digit) == 3:
        return check_symbols(x, y, engine_schematic) or  check_symbols(x, y-1, engine_schematic) or  check_symbols(x, y-2, engine_schematic)
    

current_digit = ""
stored_numbers = []

for x, line in enumerate(engine_schematic):
    # print(f"Line {x} --------------------------------------------------------------------------------------------")
    for y, char in enumerate(line):
        if char.isdigit():
            current_digit += str(char)
            condition_y = not line[y+1].isdigit() if y+1<len(line) else True
            if condition_y: # check if next char is not a digit then nr is over
                # print("checking potential nr", current_digit, check_full_number_symbol(current_digit, x, y, engine_schematic))
                if  check_full_number_symbol(current_digit, x, y, engine_schematic) == True:
                    # print("appending", current_digit, " which is close to a symbol. next char is not a digit")
                    stored_numbers.append(int(current_digit))
                else:
                    #print("skipping number", current_digit, "char", char, "pos", x, y)
                    pass
                current_digit = ""

print(sum(stored_numbers))

539637


# Part 2

In [106]:
def read_matrix_from_file(file_path):
    with open(file_path, 'r') as file:
        input_text = file.read()
        lines = input_text.strip().split('\n')
        matrix = [list(line) for line in lines]
        numpy_matrix = np.array(matrix)
        return numpy_matrix

engine_schematic = read_matrix_from_file("day3_input.txt")
special_characters = set("!@#$%^&*()_+{}[]:;<>,?/\\|=~-")

def check_symbols_gear(x, y, engine_schematic):
    x_max = engine_schematic.shape[0]
    y_max = engine_schematic.shape[1]

    x_upper = x+2 if x+2 <= x_max else x+1
    y_upper = y+2 if y+2 <= y_max else y+1
    x_lower = x-1 if x >= 1 else x
    y_lower = y-1 if y >= 1 else y

    neighbors = set(engine_schematic[x_lower:x_upper, y_lower:y_upper].flatten())
    # print("pos", x, y , "neighbors", engine_schematic[x_lower:x_upper, y_lower:y_upper])

    if len(neighbors&set("*"))>0:
        return True
    else:
        return False
    
def check_full_number_symbol_gear(current_digit, x, y, engine_schematic):
    if len(current_digit) == 1:
        return check_symbols_gear(x, y, engine_schematic)
    elif len(current_digit) == 2:
        return check_symbols_gear(x, y, engine_schematic) or  check_symbols_gear(x, y-1, engine_schematic)
    elif len(current_digit) == 3:
        return check_symbols_gear(x, y, engine_schematic) or  check_symbols_gear(x, y-1, engine_schematic) or  check_symbols_gear(x, y-2, engine_schematic)
    

def find_gear(x, y, engine_schematic): # Check in the neighbors of the star if there are numbers that can be summed
    x_max = engine_schematic.shape[0]
    y_max = engine_schematic.shape[1]

    window = 3
    x_upper = min(x_max, x+2)
    y_upper = min(y_max, y+window+1)
    x_lower = max(0, x-1)
    y_lower = max(0, y-window)

    neighbors = engine_schematic[x_lower:x_upper, y_lower:y_upper]
    neighbors = np.char.replace(neighbors, "*", ".")
    neighbors[1,3]="*"

    ## Avoid other stars that are not in the initial position:
    current_digit = ""
    stored_numbers = []
    for x, line in enumerate(neighbors): ## Looping across neighbors window
        for y, char in enumerate(line):
            if char.isdigit():
                current_digit += str(char)
                condition_y = not line[y+1].isdigit() if y+1<len(line) else True
                if condition_y: # check if next char is not a digit then nr is over
                    if  check_full_number_symbol_gear(current_digit, x, y, neighbors) == True:
                        # print("appending", current_digit, " which is close to a symbol. next char is not a digit")
                        stored_numbers.append(int(current_digit))
                    else:
                        # print("skipping number", current_digit, "char", char, "pos", x, y)
                        pass
                    current_digit = ""

    # print("*close ", neighbors)
    if len(stored_numbers)==2:
        # print("Multiplying ", stored_numbers[0], stored_numbers[1])
        gear_ratio = stored_numbers[0]*stored_numbers[1]
    else:
        # print("Not multipling ", stored_numbers)
        gear_ratio = 0
        
    return gear_ratio


prods_list_mine = []
sum_gear_ratio = 0
for x, line in enumerate(engine_schematic):
    # print(f"Line {x+1} --------------------------------------------------------------------------------------------")
    for y, char in enumerate(line):
        if char == "*":
            # print("star in pos", x, y)
            sum_gear_ratio+=find_gear(x, y, engine_schematic)
            if find_gear(x, y, engine_schematic)!=0:
                prods_list_mine +=[find_gear(x, y, engine_schematic)]

print(sum_gear_ratio)

    

82818007
